# Transfer Learning and Fine Tuning

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and tune dense layers for the classification of digits [5..9].

In [ ]:
import numpy as np
import datetime
import os
np.random.seed(1337)  # for reproducibility

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from numpy import nan

### Settings

In [ ]:
batch_size = 128
nb_classes = 5
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [ ]:
input_shape = (img_rows, img_cols, 1) # w,h,c

## Dataset Preparation

In [ ]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
X_train = np.expand_dims(X_train, axis=-1)
print(X_train.shape)
X_test = np.expand_dims(X_test, axis=-1)
print(X_test.shape)
print(y_test.shape)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
generated_images = ImageDataGenerator(
    validation_split=0.2)
generated_test = ImageDataGenerator(
    validation_split=0.0)

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(10000,)


In [ ]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  padding='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

In [ ]:
def train_model(model, loadid, X_train, X_test, Y_train, Y_test, nb_classes):

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    # convert y to one hot encoding
    Y_train = utils.to_categorical(Y_train, nb_classes)
    Y_test = utils.to_categorical(Y_test, nb_classes)

    batch_size=128
    train_gen = generated_images.flow(X_train, Y_train, batch_size=batch_size, shuffle=True, subset='training')
    val_gen = generated_images.flow(X_train, Y_train, batch_size=batch_size, shuffle=True, subset='validation')
    test_gen = generated_test.flow(X_test, Y_test, batch_size=1, shuffle=False)

    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    if loadid > 0:
        model.load_weights("Best-model0.h5")

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])

    # 儲存最好的網路模型權重

    model_mckp = keras.callbacks.ModelCheckpoint("Best-model{}.h5".format(loadid),
                                             monitor='val_categorical_accuracy',
                                             save_best_only=True,
                                             mode='max')
    model.fit_generator(generator=train_gen,
                        steps_per_epoch = X_train.shape[0]*0.8//batch_size,
                        validation_data=val_gen,
                        validation_steps= X_train.shape[0]*0.2//batch_size,
                        epochs=50,
                        callbacks=[model_mckp]
                        )
    model.load_weights("Best-model{}.h5".format(loadid))
    loss, acc = model.evaluate(test_gen, steps=X_test.shape[0])
    print('\nModel-1 Accuracy: {}%'.format(acc))


#Train a simple convnet on the MNIST dataset the first 5 digits [0..4]

In [ ]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model, 0, X_train_lt5, X_test_lt5, y_train_lt5, y_test_lt5, nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples


<ipython-input-7-56229f034ff9>:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_gen,


Epoch 1/50
191/191 [==============================] - 7s 25ms/step - loss: 0.7320 - categorical_accuracy: 0.7399 - val_loss: 0.2591 - val_categorical_accuracy: 0.9176
Epoch 2/50
191/191 [==============================] - 4s 20ms/step - loss: 0.2985 - categorical_accuracy: 0.9066 - val_loss: 0.1974 - val_categorical_accuracy: 0.9403
Epoch 3/50
191/191 [==============================] - 3s 17ms/step - loss: 0.2421 - categorical_accuracy: 0.9261 - val_loss: 0.1691 - val_categorical_accuracy: 0.9476
Epoch 4/50
191/191 [==============================] - 4s 22ms/step - loss: 0.2125 - categorical_accuracy: 0.9351 - val_loss: 0.1478 - val_categorical_accuracy: 0.9520
Epoch 5/50
191/191 [==============================] - 3s 18ms/step - loss: 0.1886 - categorical_accuracy: 0.9426 - val_loss: 0.1448 - val_categorical_accuracy: 0.9556
Epoch 6/50
191/191 [==============================] - 4s 22ms/step - loss: 0.1736 - categorical_accuracy: 0.9459 - val_loss: 0.1347 - val_categorical_accuracy: 0.959

#**Freeze convolutional layers and transfer learn** dense layers for the classification of digits [5..9].

In [ ]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model, 1, X_train_gte5, X_test_gte5, y_train_gte5, y_test_gte5, nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/50


<ipython-input-7-56229f034ff9>:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_gen,


183/183 [==============================] - 3s 15ms/step - loss: 1.8758 - categorical_accuracy: 0.4733 - val_loss: 0.9107 - val_categorical_accuracy: 0.6866
Epoch 2/50
183/183 [==============================] - 3s 16ms/step - loss: 0.9843 - categorical_accuracy: 0.6408 - val_loss: 0.7169 - val_categorical_accuracy: 0.7536
Epoch 3/50
183/183 [==============================] - 3s 14ms/step - loss: 0.8728 - categorical_accuracy: 0.6830 - val_loss: 0.6531 - val_categorical_accuracy: 0.7717
Epoch 4/50
183/183 [==============================] - 3s 15ms/step - loss: 0.8235 - categorical_accuracy: 0.7011 - val_loss: 0.6142 - val_categorical_accuracy: 0.7832
Epoch 5/50
183/183 [==============================] - 3s 17ms/step - loss: 0.8008 - categorical_accuracy: 0.7065 - val_loss: 0.5928 - val_categorical_accuracy: 0.7922
Epoch 6/50
183/183 [==============================] - 2s 13ms/step - loss: 0.7774 - categorical_accuracy: 0.7168 - val_loss: 0.5776 - val_categorical_accuracy: 0.8003
Epoch 7/5

#**UNfreeze convolutional layers and fine-tune** whole model for the classification of digits [5..9].

In [ ]:
# UNfreeze feature layers and rebuild model (Fine tune)
for l in feature_layers:
    l.trainable = True

# Fine tune: train dense layers for new classification task [5..9]
train_model(model, 2, X_train_gte5, X_test_gte5, y_train_gte5, y_test_gte5, nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/50


<ipython-input-7-56229f034ff9>:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_gen,


183/183 [==============================] - 5s 20ms/step - loss: 1.1569 - categorical_accuracy: 0.6319 - val_loss: 0.5192 - val_categorical_accuracy: 0.8252
Epoch 2/50
183/183 [==============================] - 4s 20ms/step - loss: 0.5948 - categorical_accuracy: 0.7969 - val_loss: 0.3401 - val_categorical_accuracy: 0.8894
Epoch 3/50
183/183 [==============================] - 3s 16ms/step - loss: 0.4478 - categorical_accuracy: 0.8510 - val_loss: 0.2702 - val_categorical_accuracy: 0.9141
Epoch 4/50
183/183 [==============================] - 4s 23ms/step - loss: 0.3911 - categorical_accuracy: 0.8695 - val_loss: 0.2354 - val_categorical_accuracy: 0.9233
Epoch 5/50
183/183 [==============================] - 3s 16ms/step - loss: 0.3496 - categorical_accuracy: 0.8868 - val_loss: 0.2130 - val_categorical_accuracy: 0.9286
Epoch 6/50
183/183 [==============================] - 3s 17ms/step - loss: 0.3120 - categorical_accuracy: 0.8964 - val_loss: 0.1965 - val_categorical_accuracy: 0.9339
Epoch 7/5

Reference: https://github.com/keras-team/keras/tree/master/examples